# Compare between scenarios

## 1. Prepare Base Config
Prepare the base configuration based on the function in plot_KPIs

In [ ]:
import sys
import os
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from pypsa.plot import add_legend_circles, add_legend_patches
import cartopy.crs as ccrs
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import pypsa
from matplotlib import rc
import yaml

config = {}
for c in ["kpi","default","form"]:
    with open(f"../config/config.{c}.yaml") as file:
        config_append = yaml.safe_load(file)

    config.update(config_append)

from matplotlib.colors import Normalize
import country_converter as coco

cc = coco.CountryConverter()

rc('axes', **{'edgecolor': 'None', 'titlesize': 18,'titleweight': 'bold', "labelsize": 14})
rc('figure', **{'edgecolor': 'None'})
rc('patch', **{'edgecolor': 'None'})
rc('savefig', **{'edgecolor': 'None'})
rc('legend', **{'fontsize': 12, 'title_fontsize': 12})
rc('xtick', **{'labelsize': 12})
rc('ytick', **{'labelsize': 12})
# plt.style.use(["ggplot"])

relative_path = '../scripts'
absolute_path = os.path.abspath(relative_path)
sys.path.append(absolute_path)

from scripts.plot_power_network import load_projection, test_and_enable_latex
from scripts.plot_summary import preferred_order
from scripts.plot_validation_cross_border_flows import sort_one_country, color_country

from scripts.plot_KPIs import *

power_generation_tech = pd.Index([
    "solar rooftop",
    "Solar",
    "solar-hsat",
    "Onshore Wind",
    "Offshore Wind (DC)",
    "Offshore Wind (AC)",
    "Offshore Wind (Floating)",
    "Run of River",
    "Reservoir & Dam",
    "Open-Cycle Gas",
    "OCGT",
    "Combined-Cycle Gas",
    "CCGT",
    "nuclear",
    "oil",
    "lignite",
    "coal",
    "urban central CHP",
    "urban central CHP CC",
    "urban central solid biomass CHP",
    "urban central solid biomass CHP CC",
])

tech_colors_custom = {
    "H2 Electrolysis": "#ff29d9",
    "SMR": "#f073da",
    "SMR CC": "#c251ae",
    "Fischer-Tropsch": "#25c49a",
    "H2 Fuel Cell": "#2d8077",
    "H2 for industry": "#cd4f41",
    "H2 for shipping": "#238fc4",
    "Sabatier": "#de9e46",
    "methanation": "#de9e46",
    "Reservoir \& Dam": '#298c81',
    "Reservoir & Dam": '#298c81',
    "Solar Thermal": '#d7b24c',
    "Solar (HSAT)": "#fdb915",
    "Solar PV (utility)": "#f9d002",
    "Solar PV (rooftop)": '#ffea80',
    'Offshore Wind': "#6895dd",
    'Offshore Wind (AC)': "#6895dd",
    'Offshore Wind (DC)': "#74c6f2",
    'Offshore Wind (Floating)': "#b5e2fa",
    "Onshore Wind": "#235ebc",
    "Run of River": '#3dbfb0',
    "residential urban decentral gas boiler": "#de9e46",
    "Hydroelectricity": '#298c81',
    "Gas Turbine": '#a85522',
}

pretty_gen = {
    "onwind": "Onshore Wind",
    "offwind-ac": "Offshore Wind (AC)",
    "offwind-dc": "Offshore Wind (DC)",
    "offwind-float": "Offshore Wind (Floating)",
    "solar": "Solar",
}

# choose pretty names
pretty_names = {
    "lair": "Liquid Air energy storage",
    "pair": "Adiabatic CAES",
    "vanadium": "Vanadium-Redox Battery Storage",
    "li-ion battery": "Li-Ion Battery Storage",
    "li-ion battery 1h": "Li-Ion Battery Storage 1h",
    "li-ion battery 2h": "Li-Ion Battery Storage 2h",
    "li-ion battery 4h": "Li-Ion Battery Storage 4h",
    "li-ion battery 8h": "Li-Ion Battery Storage 8h",
    "iron-air battery": "Iron-Air Battery Storage",
    "H2 Electrolysis": "H2 electrolysis",
    "H2 Fuel Cell": "H2 fuel cell",
    "Offshore Wind (AC)": "Offshore Wind", # Special Case: aggregate all offshore wind to one
    "Offshore Wind (DC)": "Offshore Wind",
    "Offshore Wind (Floating)": "Offshore Wind",
    "offshore wind (AC)": "Offshore Wind",
    "offshore wind (DC)": "Offshore Wind",
    "offwind-ac": "Offshore Wind",
    "offwind-dc": "Offshore Wind",
    "offwind-float": "Offshore Wind",
    "offshore wind": "Offshore Wind",
    "onwind": "Onshore Wind",
    "onshore wind": "Onshore Wind",
    "solar PV": "Solar PV (utility)", # Special Case: aggregate pv to utility and rooftop
    "solar": "Solar PV (utility)",
    "Solar": "Solar PV (utility)",
    "solar-hsat": "Solar PV (utility)", 
    "solar rooftop": "Solar PV (rooftop)",
    "hydro": "Hydroelectricity",
    "Reservoir & Dam": "Hydroelectricity", #"Reservoir \& Dam", # Note, revert this back to /&
    "hydroelectricity": "Hydroelectricity", # Special Case: aggregate ror and hydro to hydroelectricty
    "ror": "Hydroelectricity",
    "Run of River": "Hydroelectricity",
    "uranium": "Uranium",
    "oil emissions": "Oil emissions",
    "process emissions": "Process emissions",
    "process emissions CC": "Process emissions CC",
    "gas": "Methane",
    "oil": "Oil",
    "oil primary": "Oil",
    "oil boiler": "Oil boiler",
    "gas boiler": "Gas boiler",
    "nuclear": "Nuclear",
    "coal": "Coal", # Special Case: aggregate lignite with coal
    "lignite": "Coal", 
    "Lignite": "Coal",
    "Combined-Cycle Gas": "Gas Turbine", # Special Case: aggregate CCGT with OCGT
    "Open-Cycle Gas": "Gas Turbine",
    "CCGT": "Gas Turbine",
    "OCGT": "Gas Turbine",
    "land transport oil": "Land transport oil demand",
    "land transport EV": "Land transport EV",
    "heat": "Heat demand",
    "electricity": "Electricity demand",
    "co2": "CO2 emissions",
    "biomass boiler": "Biomass boiler",
    "hot water storage": "Hot water storage",
    "resistive heater": "Resistive heater",
    "electricity distribution grid": "Electricity distribution grid",
    "transmission lines": "Transmission lines",
    "V2G": "Vehicle-to-Grid",
}

sector_colors = {
    "Power Sector": '#ffbf2b',
    "Transport Sector": '#baf238',
    "Heating Sector": '#db6a25',
    "CCUS": '#d2d2d2',
    "Primary Fuel": '#692e0a',
    "Other": 'black'
}

sector_names = {
    "Iron-Air Battery Storage": "Power Sector",
    "Li-Ion Battery Storage": "Power Sector",
    "Li-Ion Battery Storage 1h": "Power Sector",
    "Li-Ion Battery Storage 2h": "Power Sector",
    "Li-Ion Battery Storage 4h": "Power Sector",
    "Li-Ion Battery Storage 8h": "Power Sector",
    "Vanadium Redox Flow battery storage": "Power Sector",
    "Liquid Air energy storage": "Power Sector",
    "Adiabatic CAES": "Power Sector",
    "lair": "Power Sector",
    "vanadium": "Power Sector",
    "pair": "Power Sector",
    "H2 Fuel Cell": "Power Sector",
    "H2 Store": "Power Sector",
    "H2 Electrolysis": "Power Sector",
    "Pumped Hydro Storage": "Power Sector",
    "solar rooftop": "Power Sector",
    "Solar": "Power Sector",
    "solar-hsat": "Power Sector",
    "Onshore Wind": "Power Sector",
    "Offshore Wind (DC)": "Power Sector",
    "Offshore Wind (AC)": "Power Sector",
    "Offshore Wind (Floating)": "Power Sector",
    "Run of River": "Power Sector",
    "Reservoir & Dam": "Power Sector",
    "Open-Cycle Gas": "Power Sector",
    "Combined-Cycle Gas": "Power Sector",
    "nuclear": "Power Sector",
    "AC": "Power Sector",
    "DC": "Power Sector",
    "electricity distribution grid": "Power Sector",
    "H2 pipeline": "Pipeline Infrastructure",
    "H2 pipeline retrofitted": "Pipeline Infrastructure",
    "gas pipeline": "Pipeline Infrastructure",
    "gas pipeline new": "Pipeline Infrastructure",
    "BEV charger": "Transport Sector",
    "EV battery": "Transport Sector",
    "V2G": "Transport Sector",
    "land transport oil": "Transport Sector",
    "co2": "Primary Fuel",
    "co2 stored": "CCUS",
    "co2 sequestered": "CCUS",
    "DAC": "CCUS",
    "SMR CC": "Primary Fuel",
    "SMR": "Primary Fuel",
    "Sabatier": "Primary Fuel",
    "oil refining": "Primary Fuel",
    "oil primary": "Primary Fuel",
    "oil": "Primary Fuel",
    "methanol": "Primary Fuel",
    "gas": "Primary Fuel",
    "lignite": "Primary Fuel",
    "coal": "Primary Fuel",
    "uranium": "Primary Fuel",
    "biogas": "Primary Fuel",
    "biogas to gas": "Primary Fuel",
    "solid biomass": "Primary Fuel",
}

preferred_order = pd.Index([
    "Iron-Air Battery Storage",
    "Li-Ion Battery Storage",
    "lair",
    "vanadium",
    "pair",
    "H2 Fuel Cell",
    "H2 Store",
    "H2 Electrolysis",
    "Pumped Hydro Storage",
    "solar rooftop",
    "Solar",
    "solar-hsat",
    "Onshore Wind",
    "Offshore Wind (DC)",
    "Offshore Wind (AC)",
    "Offshore Wind (Floating)",
    "Run of River",
    "Reservoir & Dam",
    "Open-Cycle Gas",
    "Combined-Cycle Gas",
    "nuclear",
    "AC",
    "DC",
    "electricity distribution grid",
    "BEV charger",
    "EV battery",
    "V2G",
    "land transport oil",
    "Solar Thermal"
    "Heat Pump"
    "Resistive Heater"
    "Gas boiler"
    "Oil boiler"
    "Water Tanks"
    "heat"
    "CHP CC"
    "CHP"
    "co2",
    "co2 stored",
    "co2 sequestered",
    "DAC",
    "SMR CC",
    "SMR",
    "Sabatier",
    "oil refining",
    "oil primary",
    "oil",
    "methanol",
    "gas",
    "lignite",
    "coal",
    "uranium"
])

def calculate_capacity(network, countries, kpi_param):
    
    n = network.copy()
    
    stats = kpi_param.get("stats","optimal")
    storage = kpi_param.get("storage",None)

    if stats == "install":
        df = pd.DataFrame(n.statistics.installed_capacity(groupby=["country", "carrier"], storage = storage))
    elif stats == "optimal":
        df = pd.DataFrame(n.statistics.optimal_capacity(groupby=["country", "carrier"], storage = storage))
    elif stats == "expand":
        if storage == True:
            df = pd.DataFrame(n.statistics.optimal_capacity(groupby=["country", "carrier"], storage = True)).subtract(pd.DataFrame(n.statistics.installed_capacity(groupby=["country", "carrier"], storage = True)), fill_value=0)
        else:
            df = pd.DataFrame(n.statistics.expanded_capacity(groupby=["country", "carrier"]))
    df = df.reset_index(["country","component"])
        
    # convert links power gen techs capacity from MWth to MWel by multiplying by efficiency
    links_i = df.query("component == 'links' and carrier in @power_generation_tech").index
    power_ge_links_eff = n.links.query("carrier in @links_i").groupby("carrier").first().efficiency
    df["efficiency"] = df.index.map(power_ge_links_eff)
    df.loc[links_i, 0] *= df.loc[links_i].efficiency
    
    df.loc[~df.country.isin(countries),"country"] = "EU"
    
    df = pd.DataFrame(df.groupby(["country","carrier"])[0].sum()).unstack("country")
    df.columns = df.columns.get_level_values("country")

    # convert from MWh to GWh
    df = df / 1e3

    return df

def calculate_curtailment(network, countries):
    n = network.copy()
    
    df = pd.DataFrame(n.statistics.curtailment(groupby=["bus","country", "carrier"]))
    df = df.reset_index(["country"])
    df.loc[~df.country.isin(countries),"country"] = "EU"

    df = pd.DataFrame(df.groupby(["country","bus","carrier"])[0].sum()).unstack(["country","bus"])
    elec_generation = n.generators.carrier.replace(pretty_gen).to_frame().query(
        "not carrier.str.contains('solar thermal')").carrier.unique()
    elec_i = pd.Index(set(df.index.intersection(elec_generation)))
    df = df.loc[elec_i, :]

    # convert from MW to GW
    df = df / 1e3
    
    return df

def filter_and_rename(network, df, filter_scheme = {}, carrier_filter = None, group_carrier = None):
    n = network.copy()

    # Replace countries with short country names
    short_name = cc.convert(names = df.columns, src = 'ISO2', to = 'name_short', not_found=None)
    if len(df.columns) == 1:
        df.columns = [short_name]
    else:
        df.columns = short_name

    # convert to nice name
    df["nice_name"] = list(pd.Series(df.index).replace(n.carriers.nice_name))
    df = df.set_index("nice_name")

    # Create your own filter
    stats = pd.DataFrame(n.statistics.energy_balance(groupby=["bus", "carrier", "bus_carrier"]))

    if carrier_filter == "electricity":
        carrier = [k for k,v in sector_names.items() if v == "Power Sector"]
        df = df.loc[df.index.isin(carrier),:]

    elif carrier_filter == "electricity+":
        carrier = stats[stats.index.get_level_values("bus_carrier").isin(["AC"])].index.get_level_values("carrier").unique()
        carrier = carrier.append(stats.filter(like="water tanks",axis=0).index.get_level_values("carrier").unique())
        carrier = carrier.append(stats.filter(like="EV battery",axis=0).index.get_level_values("carrier").unique())
        df = df.loc[df.index.isin(carrier),:]

    elif carrier_filter in filter_scheme:
        df = df.loc[df.index.isin(filter_scheme[carrier_filter]),:]

    # Group carrier
    if group_carrier == "pretty":
        df = df.rename(index=pretty_names).groupby(["nice_name"]).sum()
    elif group_carrier == "sector": 
        df = df.rename(index=sector_names).groupby(["nice_name"]).sum()

    to_drop = df.loc[abs(df.T.sum()) < 0.001,:].index
    # An exception if Iron Air Battery Storage is still in
    if 'Iron-Air Battery Storage' in to_drop:
        to_drop = to_drop.drop('Iron-Air Battery Storage')
    df = df.drop(to_drop)
    
    # Sort the index by preferred order and set color
    colors = n.carriers.set_index("nice_name").color.where(lambda s: s != "", "green")
    
    if group_carrier == "pretty":
        new_index = preferred_order_pretty_names_reversed.intersection(df.index).append(
            df.index.difference(preferred_order_pretty_names)
        )
        colors = colors.rename(index=pretty_names)
        colors.update(tech_colors)
        colors = colors[~colors.index.duplicated(keep='first')]
        
    elif group_carrier == "sector": 
        new_index = preferred_order_sector_names_reversed.intersection(df.index).append(
            df.index.difference(preferred_order_sector_names)
        )
        colors = sector_colors
    else:
        new_index = preferred_order_reversed.intersection(df.index).append(
            df.index.difference(preferred_order)
        )

    df["color"] = [colors[i] for i in df.index]

    return df.loc[new_index,:]

## 2. Importing and preparing the scenarios

In [ ]:
def extract_scenarios(scenarios, dagg_lion=False):
# Creating the dictionary
    scenarios_network = {}
    
    for scenario in scenarios:
        if 'delay' in scenario:
            postnetwork = "base_s_52_lv1.012___2035.nc"
        else:
            postnetwork = "base_s_52_lv1.11___2035.nc"

        try:
            n = pypsa.Network(f"../results/scenarios/{scenario}/postnetworks/{postnetwork}")
            scenarios_network.update({scenario:n})
        except:
            print("The " + scenario + " import from netCDF was unsuccessful.")

    return scenarios_network

def extract_durations_scenarios(n, carrier="li-ion battery"):

    scenarios_network = {}

    for scenario, n in n_scenarios.items():
        n_duration = n.copy()
        
        # prepare combining the carrier name and its duration
        carrier_i = n_duration.storage_units.filter(like=carrier, axis=0).index
        max_hours = n_duration.storage_units.loc[carrier_i, "max_hours"].apply(lambda x: str(round(x)) + "h")
    
        logger.info(f"Disaggregating {carrier} by adding {max_hours.unique()} in {scenario}")
        
        # add the carriers and nice name
        for d in max_hours.unique():
            new_carrier = carrier + " " + d
            n_duration.carriers.loc[new_carrier,:] = n_duration.carriers.loc[carrier,:]
            n_duration.carriers.loc[new_carrier,"nice_name"] = n_duration.carriers.loc[carrier,"nice_name"] + " " + d
            if new_carrier in tech_colors:
                n_duration.carriers.loc[new_carrier,"color"] = tech_colors[new_carrier]
        
        n_duration.storage_units.loc[carrier_i, "carrier"] += " " + max_hours[carrier_i]

        scenarios_network.update({scenario:n_duration})

    return scenarios_network

baseline_list = ['baseline-nomds','baseline-mds', 'mid-capex-mds','low-capex-mds']
cy2012_list = ['cy2012-nomds','cy2012-mds','cy2012-mid-capex-mds','cy2012-low-capex-mds']
cy1996_list = ['cy1996-nomds','cy1996-mds']
cy2010_list = ['cy2010-nomds','cy2010-mds','cy2010-mid-capex-mds','cy2010-low-capex-mds']
nogasppde_list = ['nogasppde-nomds','nogasppde-mid-capex-mds','nogasppde-low-capex-mds']
delayed_nep_list = ['delayed-nep-nomds','delayed-nep-mid-capex-mds','delayed-nep-low-capex-mds']
gas_price_peak_list = ['gasprice-peak-nomds','gasprice-peak-mid-capex-mds', 'gasprice-peak-low-capex-mds']

high_capex_list = ['baseline-nomds','baseline-mds','cy2012-nomds','cy2012-mds','cy1996-nomds','cy1996-mds','cy2010-nomds','cy2010-mds']
mid_capex_list = ['mid-capex-mds','cy2012-mid-capex-mds','cy2010-mid-capex-mds','nogasppde-mid-capex-mds','delayed-nep-mid-capex-mds','gasprice-peak-mid-capex-mds']
low_capex_list = ['low-capex-mds','cy2012-low-capex-mds','cy2010-low-capex-mds','nogasppde-low-capex-mds','delayed-nep-low-capex-mds','gasprice-peak-low-capex-mds']

# Order the scenarios for overview comparison
scenarios_list = baseline_list + cy2012_list + cy1996_list + cy2010_list + nogasppde_list + delayed_nep_list + gas_price_peak_list

# Extract the scenarios
n_scenarios = extract_scenarios(scenarios_list)
storylines_sorted = nogasppde_list + cy1996_list + delayed_nep_list + baseline_list + gas_price_peak_list + cy2012_list + cy2010_list

# Prepare the names and the orderings
for scenario, n in n_scenarios.items():
    pretty_names_n = expanded_pretty_names(n, pretty_names)
    sector_names_n = expanded_sector_names(n, sector_names)
    
    pretty_names.update(pretty_names_n)
    sector_names.update(sector_names_n)
    logger.info(f"Cycling names in {scenario}")
    
# defined preferred order under all naming schemes
preferred_order_reversed = preferred_order[::-1]
preferred_order_pretty_names = pd.Index(pd.Series(preferred_order).apply(lambda x: pretty_names[x] if x in pretty_names.keys() else x).drop_duplicates(keep = 'first'))
preferred_order_pretty_names_reversed = preferred_order_pretty_names[::-1]

preferred_order_sector_names = pd.Index(pd.Series(preferred_order).apply(lambda x: sector_names[x] if x in sector_names.keys() else x).drop_duplicates(keep = 'first'))
preferred_order_sector_names_reversed = preferred_order_sector_names[::-1]

# define tech colors and update with custom
tech_colors = config["plotting"]["tech_colors"]
tech_colors.update(tech_colors_custom)
tech_colors = {(pretty_gen[k] if k in pretty_gen else k):v  for (k,v) in tech_colors.items()}

# test run latex compilation (This results in a generic plot)
config_kpi = config["kpi"]
if config_kpi.get("enable_latex", False): test_and_enable_latex()

# extract the duration from lithium ion
n_scenarios = extract_durations_scenarios(n_scenarios)

# Divide the scenarios based on the list above
n_scenarios_baseline = {key: n_scenarios[key] for key in baseline_list}
n_scenarios_cy2012 = {key: n_scenarios[key] for key in ["baseline-nomds"] + cy2012_list}
n_scenarios_cy1996 = {key: n_scenarios[key] for key in ["baseline-nomds"] + cy1996_list}
n_scenarios_cy2010 = {key: n_scenarios[key] for key in ["baseline-nomds"] + cy2010_list}
n_scenarios_nogasppde = {key: n_scenarios[key] for key in ["baseline-nomds"] + nogasppde_list}
n_scenarios_delayed_nep = {key: n_scenarios[key] for key in ["baseline-nomds"] + delayed_nep_list}
n_scenarios_gas_price_peak = {key: n_scenarios[key] for key in ["baseline-nomds"] + gas_price_peak_list}

n_scenarios_high_capex = {key: n_scenarios[key] for key in high_capex_list}
n_scenarios_mid_capex = {key: n_scenarios[key] for key in mid_capex_list}
n_scenarios_low_capex = {key: n_scenarios[key] for key in low_capex_list}
n_scenarios_storylines = {key: n_scenarios[key] for key in storylines_sorted}

# 3. Plotting Comparison from KPIs Script:

In [ ]:
def compare_scenarios(n_scenarios, kpi_param, filter_scheme, path, sort=False, legend=True):
    df_all = pd.DataFrame()
    
    countries = config["countries"]
    extract_param = kpi_param.get("extract",None)
    include = kpi_param.get("include",False)
    exclude = kpi_param.get("exclude",False)
    plot_kw = kpi_param.get("plot_kw",{})
    
    for scenario, n in n_scenarios.items():

        RESULTS = f"../results/scenarios/{scenario}/"
        nodal_costs=RESULTS + "csvs/nodal_costs.csv"
        nodal_capacity=RESULTS + "csvs/nodal_capacities.csv"

        if extract_param == "system cost":
            df = calculate_system_cost_csv(n, nodal_costs, countries)
        elif extract_param == "capacity":
            df = calculate_capacity_csv(n, nodal_capacity, countries)
        if extract_param == "capacity stats":
            df = calculate_capacity(n, countries, kpi_param)
        elif extract_param == "generation":
            df = calculate_generation(n, countries)
        elif extract_param == "emission":
            df = calculate_emission(n, countries)
        elif extract_param == "curtailment":
            df = calculate_curtailment(n, countries)

        df = df.T.groupby("country").sum().T

        if include:
            #logger.info(f"include only {include}")
            df = df[include]
        if exclude:
            #logger.info(f"exclude {exclude}")
            df = df.loc[:,df.columns.difference(exclude)]

        if "TW" in plot_kw.get("ylabel", ""):
            df *= 1e-3

        df = filter_and_rename(n, df, 
                               filter_scheme = filter_scheme,
                               carrier_filter = kpi_param.get("carrier_filter",None), 
                               group_carrier = kpi_param.get("group_carrier",None), 
                               )
        
        df_data = pd.DataFrame(df[df.columns.difference(['color'])].T.sum()).rename(columns={0:scenario})
        df_data["color"] = df["color"]
    
        if df_all.empty:
            df_all = df_data
        else:
            df_all = pd.concat([df_all,df_data[[scenario]]], axis=1)
            df_all["color"] = df_all["color"].combine_first(df_data["color"])

    if plot_kw.get("ylabel", "") == "%":
        plot_kw["ylabel"] = "\%"
        df_color = df_all[["color"]]
        df = df_all[df_all.columns.difference(['color'])]
        df = df/df.sum()*100
        df_all = df
        df_all["color"] = df_color["color"]
    
    if isinstance(sort, str):
        sorted_index = list(df_all.drop(columns="color").loc[[sort],:].sum().sort_values(ascending=False).index) + ["color"]
        df_all = df_all[sorted_index]
    elif isinstance(sort, bool):
        if sort:
            sorted_index = list(df_all.drop(columns="color").sum().sort_values(ascending=False).index) + ["color"]
            df_all = df_all[sorted_index]

    plot_kw["width"] = plot_kw.get("width",0.9)
    plot_kw["rot"] = plot_kw.get("rot",90)
    
    df_all.columns = df_all.columns.str.replace('-', '\n', n=1)

    if len(df_all.columns) > 4 and plot_kw["rot"] == 0:
        df_all.columns = df_all.columns.str.replace('-mds', '\nmds', n=1)

    plot_figsize = tuple(kpi_param.get("figsize",(12,9)))
    fig, ax = plt.subplots(figsize=plot_figsize)
    
    df_all.drop("color", axis=1).T.plot(
        kind="bar",
        ax=ax,
        stacked=True,
        color=df_all["color"],
        legend=False,
        **plot_kw,
    )

    ax.grid(axis="x")
    ax.set_xlabel("")
    ax.set_facecolor("white")

    if not plot_kw.get("ylim", False):
        ymin, ymax = ax.get_ylim()
        margin = abs(ymax - ymin) * 0.1
        if ymin == 0:
            ymin = margin
        ax.set_ylim(ymin - margin, ymax + margin)

    handles, labels = ax.get_legend_handles_labels()
    
    handles.reverse()
    labels.reverse()
    labels = [label.replace(' &', '').replace(' and', '') for label in labels] #NOTE: Latex hates '&' strings because its their seperator

    if legend:
        ax.legend(handles, labels, loc = "upper center", bbox_to_anchor = (0.5, -0.20), frameon = False, ncol = 2, 
                    title = "Components", title_fontproperties = {'weight':'bold'})
    col = 0
    for c in df_all.drop("color", axis=1).columns:
        data_sum = df_all[c].sum()
        ax.annotate(str(round(data_sum,1)),xy=(col,data_sum), xytext=(0, 4), 
                textcoords='offset points', ha='center', va='bottom', 
                rotation=0, color = "black", size=12)

        col += 1

    fig.savefig(path, bbox_inches="tight")

## A. Plot Electricity System Cost:

In [ ]:
kpi_param = {
    'extract': "system cost",
    'include': ["DE"],
    'carrier_filter': "electricity",
    'group_carrier': "pretty",
    'figsize': [18, 8],
    'plot_kw': {"title": "Germany electricity system cost",
                "ylim": [0,70],
                "ylabel": "Billion EUR/a",
                "width": 0.6,
               }}
title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

filter_scheme = config["kpi"].get("filter_scheme",{})

compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}overview.pdf", legend=False)
compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}sort.pdf", sort = True, legend=False)

kpi_param['plot_kw']['rot'] = 0
kpi_param['figsize'] = [7, 4]

compare_scenarios(n_scenarios_cy2012, kpi_param, filter_scheme, f"{folder}cy2012.pdf")
compare_scenarios(n_scenarios_cy1996, kpi_param, filter_scheme, f"{folder}cy1996.pdf")
compare_scenarios(n_scenarios_cy2010, kpi_param, filter_scheme, f"{folder}cy2010.pdf")
compare_scenarios(n_scenarios_baseline, kpi_param, filter_scheme, f"{folder}baseline.pdf")
compare_scenarios(n_scenarios_nogasppde, kpi_param, filter_scheme, f"{folder}nogasppde.pdf")
compare_scenarios(n_scenarios_delayed_nep, kpi_param, filter_scheme, f"{folder}delayed_nep.pdf")
compare_scenarios(n_scenarios_gas_price_peak, kpi_param, filter_scheme, f"{folder}gas_price_peak.pdf")
compare_scenarios(n_scenarios_high_capex, kpi_param, filter_scheme, f"{folder}high_capex.pdf")

kpi_param['plot_kw']['rot'] = 90

compare_scenarios(n_scenarios_mid_capex, kpi_param, filter_scheme, f"{folder}mid_capex.pdf", legend=False)
compare_scenarios(n_scenarios_low_capex, kpi_param, filter_scheme, f"{folder}low_capex.pdf", legend=False)

## B. Plot Emission:

In [ ]:
kpi_param = {
    'extract': "emission",
    'include': ["DE"],
    'carrier_filter': "electricity+",
    'group_carrier': "pretty",
    'figsize': [18, 8],
    'plot_kw': {"title": "Germany annual power sector emission",
                "ylim": [0,27],
                "ylabel": "MtCO2",
                "width": 0.6
               }}

title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

filter_scheme = config["kpi"].get("filter_scheme",{})

compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}overview.pdf", legend=False)
compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}sort.pdf", sort = True, legend=False)

kpi_param['plot_kw']['rot'] = 0
kpi_param['figsize'] = [7, 4]

compare_scenarios(n_scenarios_cy2012, kpi_param, filter_scheme, f"{folder}cy2012.pdf")
compare_scenarios(n_scenarios_cy1996, kpi_param, filter_scheme, f"{folder}cy1996.pdf")
compare_scenarios(n_scenarios_cy2010, kpi_param, filter_scheme, f"{folder}cy2010.pdf")
compare_scenarios(n_scenarios_baseline, kpi_param, filter_scheme, f"{folder}baseline.pdf")
compare_scenarios(n_scenarios_nogasppde, kpi_param, filter_scheme, f"{folder}nogasppde.pdf")
compare_scenarios(n_scenarios_delayed_nep, kpi_param, filter_scheme, f"{folder}delayed_nep.pdf")
compare_scenarios(n_scenarios_gas_price_peak, kpi_param, filter_scheme, f"{folder}gas_price_peak.pdf")
compare_scenarios(n_scenarios_high_capex, kpi_param, filter_scheme, f"{folder}high_capex.pdf")

kpi_param['plot_kw']['rot'] = 90

compare_scenarios(n_scenarios_mid_capex, kpi_param, filter_scheme, f"{folder}mid_capex.pdf", legend=False)
compare_scenarios(n_scenarios_low_capex, kpi_param, filter_scheme, f"{folder}low_capex.pdf", legend=False)

## C. Plot Curtailment

In [ ]:
kpi_param = {
    'extract': "curtailment",
    'include': ["DE"],
    'carrier_filter': "power",
    'group_carrier': "pretty",
    'figsize': [18, 8],
    'plot_kw': {"title": "Germany electricity curtailment",
                "ylim": [0,180],
                "ylabel": "TWh",
                "width": 0.6
               }}

title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

filter_scheme = config["kpi"].get("filter_scheme",{})

compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}overview.pdf", legend=False)
compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}sort.pdf", sort = True, legend=False)

kpi_param['plot_kw']['rot'] = 0
kpi_param['figsize'] = [7, 4]

compare_scenarios(n_scenarios_cy2012, kpi_param, filter_scheme, f"{folder}cy2012.pdf")
compare_scenarios(n_scenarios_cy1996, kpi_param, filter_scheme, f"{folder}cy1996.pdf")
compare_scenarios(n_scenarios_cy2010, kpi_param, filter_scheme, f"{folder}cy2010.pdf")
compare_scenarios(n_scenarios_baseline, kpi_param, filter_scheme, f"{folder}baseline.pdf")
compare_scenarios(n_scenarios_nogasppde, kpi_param, filter_scheme, f"{folder}nogasppde.pdf")
compare_scenarios(n_scenarios_delayed_nep, kpi_param, filter_scheme, f"{folder}delayed_nep.pdf")
compare_scenarios(n_scenarios_gas_price_peak, kpi_param, filter_scheme, f"{folder}gas_price_peak.pdf")
compare_scenarios(n_scenarios_high_capex, kpi_param, filter_scheme, f"{folder}high_capex.pdf")

kpi_param['plot_kw']['rot'] = 90

compare_scenarios(n_scenarios_mid_capex, kpi_param, filter_scheme, f"{folder}mid_capex.pdf", legend=False)
compare_scenarios(n_scenarios_low_capex, kpi_param, filter_scheme, f"{folder}low_capex.pdf", legend=False)

## D. Plot storage capacity
### 1. By energy capacity

In [ ]:
kpi_param = {
    'extract': "capacity stats",
    'stats': 'expand',
    'storage': True,
    'include': ["DE"],
    'carrier_filter': "storage-energy",
    'group_carrier': "pretty",
    'figsize': [18, 8],
    'plot_kw': {"title": "Germany expanded storage energy capacity",
                "ylabel": "GWh",
                "width": 0.6
               }}

title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

filter_scheme = config["kpi"].get("filter_scheme",{})

compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}overview.pdf", legend=False)
compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}sort.pdf", sort = True, legend=False)

kpi_param['plot_kw']['rot'] = 0
kpi_param['figsize'] = [7, 4]

compare_scenarios(n_scenarios_cy2012, kpi_param, filter_scheme, f"{folder}cy2012.pdf")
compare_scenarios(n_scenarios_cy1996, kpi_param, filter_scheme, f"{folder}cy1996.pdf")
compare_scenarios(n_scenarios_cy2010, kpi_param, filter_scheme, f"{folder}cy2010.pdf")
compare_scenarios(n_scenarios_baseline, kpi_param, filter_scheme, f"{folder}baseline.pdf")
compare_scenarios(n_scenarios_nogasppde, kpi_param, filter_scheme, f"{folder}nogasppde.pdf")
compare_scenarios(n_scenarios_delayed_nep, kpi_param, filter_scheme, f"{folder}delayed_nep.pdf")
compare_scenarios(n_scenarios_gas_price_peak, kpi_param, filter_scheme, f"{folder}gas_price_peak.pdf")
compare_scenarios(n_scenarios_high_capex, kpi_param, filter_scheme, f"{folder}high_capex.pdf", legend=False)

kpi_param['plot_kw']['rot'] = 90

compare_scenarios(n_scenarios_mid_capex, kpi_param, filter_scheme, f"{folder}mid_capex.pdf", legend=False)
compare_scenarios(n_scenarios_low_capex, kpi_param, filter_scheme, f"{folder}low_capex.pdf", legend=False)

### 2. By power capacity

In [ ]:
kpi_param = {
    'extract': "capacity stats",
    'include': ["DE"],
    'storage': False,
    'carrier_filter': "storage-cap",
    'group_carrier': "pretty",
    'figsize': [18, 8],
    'plot_kw': {"title": "Relative share of Germany storage power capacity",
                "ylim": [0,105],
                "ylabel": "%",
                "width": 0.6
               }}

title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}sort_iron.pdf", sort = "Iron-Air Battery Storage", legend=False)

In [ ]:
kpi_param = {
    'extract': "capacity stats",
    'include': ["DE"],
    'storage': False,
    'carrier_filter': "storage-cap",
    'group_carrier': "pretty",
    'figsize': [18, 8],
    'plot_kw': {"title": "Germany storage power capacity",
                "ylim": [0,80],
                "ylabel": "GWe",
                "width": 0.6
               }}

title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

filter_scheme = config["kpi"].get("filter_scheme",{})

compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}overview.pdf", legend=False)
compare_scenarios(n_scenarios, kpi_param, filter_scheme, f"{folder}sort.pdf", sort = True, legend=False)

kpi_param['plot_kw']['rot'] = 0
kpi_param['figsize'] = [7, 4]

compare_scenarios(n_scenarios_cy2012, kpi_param, filter_scheme, f"{folder}cy2012.pdf")
compare_scenarios(n_scenarios_cy1996, kpi_param, filter_scheme, f"{folder}cy1996.pdf")
compare_scenarios(n_scenarios_cy2010, kpi_param, filter_scheme, f"{folder}cy2010.pdf")
compare_scenarios(n_scenarios_baseline, kpi_param, filter_scheme, f"{folder}baseline.pdf")
compare_scenarios(n_scenarios_nogasppde, kpi_param, filter_scheme, f"{folder}nogasppde.pdf")
compare_scenarios(n_scenarios_delayed_nep, kpi_param, filter_scheme, f"{folder}delayed_nep.pdf")
compare_scenarios(n_scenarios_gas_price_peak, kpi_param, filter_scheme, f"{folder}gas_price_peak.pdf")
compare_scenarios(n_scenarios_high_capex, kpi_param, filter_scheme, f"{folder}high_capex.pdf", legend=False)

kpi_param['plot_kw']['rot'] = 90

compare_scenarios(n_scenarios_mid_capex, kpi_param, filter_scheme, f"{folder}mid_capex.pdf", legend=False)
compare_scenarios(n_scenarios_low_capex, kpi_param, filter_scheme, f"{folder}low_capex.pdf", legend=False)

# E. (Jupyter Notebook only) Price comparison

In [2]:
def violin_comp(data, title, output, figsize, plot_kw = {}):
    fig, ax = plt.subplots(figsize =figsize)
    
    new_index_order = list(data.columns)
    
    data_array = [data[c].to_numpy() for c in data.columns]
    
    parts = ax.violinplot(data_array, showmeans=True, showmedians=True)
    
    ax.set_ylabel("Marginal Price [€/MWh]")
    ax.grid(axis="y")

    for pc in parts['bodies']:
        pc.set_facecolor('lightgrey')
        pc.set_edgecolor('lightgrey')
        pc.set_alpha(1)

    for partname in ('cbars','cmins','cmaxes'):
        vp = parts[partname]
        vp.set_linewidth(0)
        
    vp = parts['cmeans']
    vp.set_edgecolor("black")
    vp.set_linewidth(6)

    vp = parts['cmaxes']
    print(vp)
    vp.set_edgecolor("red")
    vp.set_linewidth(6)

    vp = parts['cmedians']
    vp.set_edgecolor("blue")
    vp.set_linewidth(6)
    
    col = 1
    for c in data.columns:

        # Max data
        data_max = data[c].max()
        ax.annotate(str(round(data_max)),xy=(col,data_max), xytext=(0, 4), 
                    textcoords='offset points', ha='center', va='bottom', 
                    rotation=0, color = "red", size=12)

        # Mean data
        data_mean = data[c].mean()
        ax.annotate(str(round(data_mean)),xy=(col,data_mean), xytext=(0, 4), 
                    textcoords='offset points', ha='center', va='bottom', 
                    rotation=0, color = "black", size=12)

        # Median data
        data_median = data[c].median()
        ax.annotate(str(round(data_median)),xy=(col,data_median), xytext=(0, 4), 
                    textcoords='offset points', ha='center', va='bottom', 
                    rotation=0, color = "blue", size=12)        
        col += 1

    if plot_kw.get("legend",True):
        pypsa.plot.add_legend_patches(ax=ax, colors = ["blue","black","red"], labels = ["median","mean","max"],
                                 legend_kw = {'loc':"lower center",'bbox_to_anchor':plot_kw.get('bbox_to_anchor',(0.5, 1.0)), 'frameon':False, 'ncol':3})

    ax.set_title(title, y=1.00)

    plt.xticks(range(1,len(data.columns)+1), list(data.columns), rotation=plot_kw.get("rot",90))
    plt.grid(axis='both', color='0.95')
    ax.set_ylim(plot_kw.get("ylim",[0,700]))
    plt.savefig(output,bbox_inches="tight", pad_inches=1)

def violin_comp_grouped(data, title, output, figsize, plot_kw = {}):
    fig, ax = plt.subplots(figsize =figsize)
    
    new_index_order = list(data.columns)
    
    data_array = [data[c].to_numpy() for c in data.columns]
    
    # Configure spacing based on multi-index structure
    group_spacing = plot_kw.get("group_spacing", 2.0)  # Space between groups
    within_spacing = plot_kw.get("within_spacing", 1.0)  # Space within groups

    # Get unique top-level groups
    groups = data.columns.get_level_values(0).unique()

    # Calculate positions for each violin
    positions = []
    tick_positions = []
    tick_labels = []

    current_pos = 1
    for i, group in enumerate(groups):
        # Get columns in this group
        group_cols = [col for col in data.columns if col[0] == group]

        # Store positions for violins in this group
        group_positions = [current_pos + j * within_spacing for j in range(len(group_cols))]
        positions.extend(group_positions)

        # Store center position for group label
        if len(group_positions) > 0:
            center = sum(group_positions) / len(group_positions)
            tick_positions.append(center)
            tick_labels.append(group)
        current_pos = group_positions[-1] + group_spacing if group_positions else current_pos

    parts = ax.violinplot(data_array, positions=positions, showmeans=True, showmedians=True)
    
    ax.set_ylabel("Marginal Price [€/MWh]")
    ax.grid(axis="y")

    for pc in parts['bodies']:
        pc.set_facecolor('lightgrey')
        pc.set_edgecolor('lightgrey')
        pc.set_alpha(1)

    for partname in ('cbars','cmins','cmaxes'):
        vp = parts[partname]
        vp.set_linewidth(0)
        
    vp = parts['cmeans']
    vp.set_edgecolor("black")
    vp.set_linewidth(6)

    vp = parts['cmaxes']
    vp.set_edgecolor("red")
    vp.set_linewidth(6)

    vp = parts['cmedians']
    vp.set_edgecolor("blue")
    vp.set_linewidth(6)
    
    col = 1
    for i, c in enumerate(data.columns):
        col_pos = positions[i]  # Use position from calculated array

        # Max data
        data_max = data[c].max()
        ax.annotate(str(round(data_max)), xy=(col_pos, data_max), xytext=(0, 4),
                   textcoords='offset points', ha='center', va='bottom',
                   rotation=0, color="red", size=12)

        # Mean data
        data_mean = data[c].mean()
        ax.annotate(str(round(data_mean)),xy=(col_pos,data_mean), xytext=(0, 4),
                    textcoords='offset points', ha='center', va='bottom',
                    rotation=0, color = "black", size=12)

        # Median data
        data_median = data[c].median()
        ax.annotate(str(round(data_median)),xy=(col_pos,data_median), xytext=(0, 4),
                    textcoords='offset points', ha='center', va='bottom',
                    rotation=0, color = "blue", size=12)
        col += 1

    if plot_kw.get("legend",True):
        pypsa.plot.add_legend_patches(ax=ax, colors = ["blue","black","red"], labels = ["median","mean","max"],
                                 legend_kw = {'loc':"lower center",'bbox_to_anchor':plot_kw.get('bbox_to_anchor',(0.5, 1.0)), 'frameon':False, 'ncol':3})

    ax.set_title(title, y=1.00)

    # Only set positions for the actual violin plots
    ax.set_xticks(positions)

    # For multi-level labeling, use empty strings at the violin positions
    ax.set_xticklabels([""] * len(positions))

    # Add group labels with text annotations instead of ticks
    for i, group in enumerate(groups):
        group_cols = [col for col in data.columns if col[0] == group]
        group_positions = [positions[list(data.columns).index(col)] for col in group_cols]

        if group_positions:
            # Calculate center of group
            center = sum(group_positions) / len(group_positions)

            # Add group label as text annotation
            ax.text(center, -0.3, group, ha='center', va='top',
                    transform=ax.get_xaxis_transform(), fontsize=12, fontweight='bold')

            # Add subgroup labels directly
            for pos, col in zip(group_positions, group_cols):
                subgroup = col[1] if len(col) > 1 else ""
                ax.text(pos, -0.05, subgroup, ha='center', va='top', wrap=True, rotation=plot_kw.get("subgroup_rot", 90),
                        transform=ax.get_xaxis_transform(), fontsize=10)

    # Add more bottom margin to accommodate the labels
    plt.subplots_adjust(bottom=0.2)

    plt.grid(axis='both', color='0.95')
    ax.set_ylim(plot_kw.get("ylim",[0,700]))
    plt.savefig(output,bbox_inches="tight", pad_inches=1)

def compare_violin(n_scenarios, include, title, output, sort_by="", figsize = [18, 8], plot_kw={}):
    df_all = pd.DataFrame()

    for scenario, n in n_scenarios.items():
    
        bus = n.buses.query("carrier == 'AC'")
        marginal = (n.buses_t.marginal_price * n.loads_t.p).loc[:,bus.index].rename(columns=bus.country).T.groupby("Bus").sum()
        marginal_country = n.loads_t.p.loc[:,bus.index].rename(columns=bus.country).T.groupby("Bus").sum()
        
        df = (marginal / marginal_country).T
    
        if include:
            df = df[include]
    
        df_data = pd.DataFrame(df.T.sum()).rename(columns={0:scenario})
        df_data = df_data.reset_index(drop = True)

        df_all = pd.concat([df_all,df_data], axis=1)

    if sort_by == "mean":
        sorted_index = df_all.mean().sort_values(ascending=False).index
    elif sort_by == "max":
        sorted_index = df_all.max().sort_values(ascending=False).index
    elif sort_by == "median":
        sorted_index = df_all.median().sort_values(ascending=False).index
    elif sort_by == "span":
        sorted_index = (df_all.max() - df_all.mean()).sort_values(ascending=False).index
    else:
        sorted_index = df_all.columns

    if "nogasppde-nomds" in df_all.columns:
        print(df_all["nogasppde-nomds"].max())
    
    df_all = df_all[sorted_index]
    df_all.columns = df_all.columns.str.replace('-', '\n', n=1)

    if len(df_all.columns) > 4 and plot_kw["rot"] == 0:
        df_all.columns = df_all.columns.str.replace('-mds', '\nmds', n=1)

    if sort_by == "storylines":
        # add second layer with groups
        df_all_T = df_all.T
        df_all_T["group"] = ["Mandated German \n gas phase out", "Mandated German \n gas phase out", "Mandated German \n gas phase out",
                             "Climate year \n 1996", "Climate year \n 1996",
                             "Delayed transmission \n expansion", "Delayed transmission \n expansion", "Delayed transmission \n expansion",
                             "Climate year \n 2013", "Climate year \n 2013", "Climate year \n 2013", "Climate year \n 2013",
                             "Gas price \n sensitivity", "Gas price \n sensitivity", "Gas price \n sensitivity",
                             "Climate year \n 2012", "Climate year \n 2012", "Climate year \n 2012", "Climate year \n 2012",
                             "Climate year \n 2010", "Climate year \n 2010", "Climate year \n 2010", "Climate year \n 2010"]
        df_all_T.set_index("group", append=True, inplace=True)
        df_all_T = df_all_T.reorder_levels([1,0])
        df_all = df_all_T.T

        violin_comp_grouped(df_all, title, output, figsize, plot_kw=plot_kw)
    else:
        violin_comp(df_all, title, output, figsize, plot_kw=plot_kw)

In [ ]:
include = ["DE"]
title = "Germany electricity price graph"
output = "test.pdf"

filename = title

figsize = [18, 8]
plot_kw = {"rot":90,
          'bbox_to_anchor': (0.5, 0.95)}

folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

compare_violin(n_scenarios_storylines, include, title, f"{folder}storylines.pdf", sort_by="storylines", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios, include, title, f"{folder}overview.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios, include, title, f"{folder}mean.pdf",sort_by = "mean", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios, include, title, f"{folder}max.pdf",sort_by = "max", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios, include, title, f"{folder}span.pdf",sort_by = "span", figsize=figsize, plot_kw=plot_kw)

figsize = [7, 4]
plot_kw = {"rot":0,
          'bbox_to_anchor': (0.5, 0.9)}

compare_violin(n_scenarios_cy2012, include, title, f"{folder}cy2012.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_cy1996, include, title, f"{folder}cy1996.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_cy2010, include, title, f"{folder}cy2010.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_baseline, include, title, f"{folder}baseline.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_nogasppde, include, title, f"{folder}nogasppde.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_delayed_nep, include, title, f"{folder}delayed_nep.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_gas_price_peak, include, title, f"{folder}gas_price_peak.pdf", figsize=figsize, plot_kw=plot_kw)

plot_kw = {"rot":90,
          'bbox_to_anchor': (0.5, 0.9)}

compare_violin(n_scenarios_high_capex, include, title, f"{folder}high_capex.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_mid_capex, include, title, f"{folder}mid_capex.pdf", figsize=figsize, plot_kw=plot_kw)
compare_violin(n_scenarios_low_capex, include, title, f"{folder}low_capex.pdf", figsize=figsize, plot_kw=plot_kw)

# F. (Jupyter Notebook only) gas usage in bcm

In [ ]:
def first_known_country(row, country_col):
    for col in ["country"] + country_col:
        value = row[col]
        if pd.notna(value) and value != '':
            return value
    return "" 

def compare_gas_usage_bcm(n_scenarios, kpi_param, path, sort=False, legend=True, drop_boiler=True):
    
    df_all = pd.DataFrame()
    include = kpi_param.get("include",False)
    exclude = kpi_param.get("exclude",False)
    plot_kw = kpi_param.get("plot_kw",{})
    
    for scenario, n in n_scenarios.items():
        df = pd.DataFrame(n.statistics.energy_balance(groupby=["name","unit","bus","carrier","bus_carrier"], nice_names=False))
        df = df.assign(country=lambda x: x.index.get_level_values("bus").map(n.buses.country))
        df = df[(df.index.get_level_values("component") == "Link")]
        
        # Find the countries of links where the bus0 are the EU (for sector coupled)
        link_index = df[(df.index.get_level_values("component") == "Link") & (df.country == "")].index
        link_index_name = link_index.get_level_values("name")
        link_number_of_buses = sum(n.links.columns.str.startswith('bus'))
        link_destination_country = [f"country{num}" for num in range(1,link_number_of_buses)]
        
        for num in range(1,link_number_of_buses):
            country_values = n.links.loc[link_index_name, f"bus{num}"].map(n.buses.country)
            df.loc[link_index, f"country{num}"] = country_values.values
        
        df["country"] = df.apply(first_known_country, axis=1,country_col=link_destination_country)
        df = df.drop(columns=link_destination_country)
        
        df = df[df.index.get_level_values("bus") == "EU gas"]
        df = df.groupby(["carrier","country"]).sum().unstack("country")
        df = df.T.droplevel(0).T
        df = df*-1/9769444.44 # 1 btu = 9,769,444.44 MWh (assuming the gas being 35.17 MJ/m3)
        
        if include:
            #logger.info(f"include only {include}")
            df = df[include]
        if exclude:
            #logger.info(f"exclude {exclude}")
            df = df.loc[:,df.columns.difference(exclude)]
    
        df_data = pd.DataFrame(df.T.sum()).rename(columns={0:scenario})
    
        if df_all.empty:
            df_all = df_data
        else:
            df_all = pd.concat([df_all,df_data[[scenario]]], axis=1)
            #df_all["color"] = df_all["color"].combine_first(df_data["color"])
    
    df_all = df_all.rename(index=n.carriers.nice_name)
    df_all = df_all.rename(index=pretty_names).groupby(["carrier"]).sum()
    
    to_drop = df_all.loc[abs(df_all.T.sum()) < 0.001,:].index
    df_all = df_all.drop(to_drop)
        
    # Sort the index by preferred order and set color
    colors = n.carriers.set_index("nice_name").color.where(lambda s: s != "", "green")
    
    new_index = preferred_order_pretty_names_reversed.intersection(df_all.index).append(
        df_all.index.difference(preferred_order_pretty_names)
    )
    colors = colors.rename(index=pretty_names)
    colors.update(tech_colors)
    colors = colors[~colors.index.duplicated(keep='first')]
    colors["CHP"] = '#8a5751'
    colors["CHP CC"] = '#634643'
    
    df_all["color"] = [colors[i] for i in df_all.index]

    if drop_boiler:
        df_all = df_all.drop("Gas boiler")
    
    if isinstance(sort, str):
        sorted_index = list(df_all.drop(columns="color").loc[[sort],:].sum().sort_values(ascending=False).index) + ["color"]
        df_all = df_all[sorted_index]
    elif isinstance(sort, bool):
        if sort:
            sorted_index = list(df_all.drop(columns="color").sum().sort_values(ascending=False).index) + ["color"]
            df_all = df_all[sorted_index]
    
    plot_kw["width"] = plot_kw.get("width",0.9)
    plot_kw["rot"] = plot_kw.get("rot",90)
    
    df_all.columns = df_all.columns.str.replace('-', '\n', n=1)
    
    if len(df_all.columns) > 4 and plot_kw["rot"] == 0:
        df_all.columns = df_all.columns.str.replace('-mds', '\nmds', n=1)
    
    plot_figsize = tuple(kpi_param.get("figsize",(12,9)))
    fig, ax = plt.subplots(figsize=plot_figsize)
    
    df_all.drop("color", axis=1).T.plot(
        kind="bar",
        ax=ax,
        stacked=True,
        color=df_all["color"],
        legend=False,
        **plot_kw,
    )
    
    ax.grid(axis="x")
    ax.set_xlabel("")
    ax.set_facecolor("white")
    
    if not plot_kw.get("ylim", False):
        ymin, ymax = ax.get_ylim()
        margin = abs(ymax - ymin) * 0.1
        if ymin == 0:
            ymin = margin
        ax.set_ylim(ymin - margin, ymax + margin)
    
    handles, labels = ax.get_legend_handles_labels()
    
    handles.reverse()
    labels.reverse()
    labels = [label.replace(' &', '').replace(' and', '') for label in labels] #NOTE: Latex hates '&' strings because its their seperator
    
    if legend:
        ax.legend(handles, labels, loc = "upper center", bbox_to_anchor = (0.5, -0.20), frameon = False, ncol = 2, 
                    title = "Components", title_fontproperties = {'weight':'bold'})
    col = 0
    for c in df_all.drop("color", axis=1).columns:
        data_sum = df_all[c].sum()
        ax.annotate(str(round(data_sum,1)),xy=(col,data_sum), xytext=(0, 4), 
                textcoords='offset points', ha='center', va='bottom', 
                rotation=0, color = "black", size=12)
    
        col += 1
    
    fig.savefig(path, bbox_inches="tight")

In [ ]:
kpi_param = {
    'include': ["DE"],
    'figsize': [18, 8],
    'plot_kw': {"title": "Germany annual power sector gas consumption",
                "ylim": [0,20],
                "ylabel": "bcm",
                "width": 0.6
               }}

title = kpi_param['plot_kw']["title"]
folder = title + "/"
if not os.path.exists(title):
    os.mkdir(title)

compare_gas_usage_bcm(n_scenarios, kpi_param, f"{folder}overview.pdf", legend=False, drop_boiler=True)
compare_gas_usage_bcm(n_scenarios, kpi_param, f"{folder}sort.pdf", sort = True, legend=False, drop_boiler=True)

kpi_param['plot_kw']['rot'] = 0
kpi_param['figsize'] = [7, 4]

compare_gas_usage_bcm(n_scenarios_cy2012, kpi_param, f"{folder}cy2012.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_cy1996, kpi_param, f"{folder}cy1996.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_cy2010, kpi_param, f"{folder}cy2010.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_baseline, kpi_param, f"{folder}baseline.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_nogasppde, kpi_param, f"{folder}nogasppde.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_delayed_nep, kpi_param, f"{folder}delayed_nep.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_gas_price_peak, kpi_param, f"{folder}gas_price_peak.pdf", drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_high_capex, kpi_param, f"{folder}high_capex.pdf", legend=False, drop_boiler=True)

kpi_param['plot_kw']['rot'] = 90

compare_gas_usage_bcm(n_scenarios_mid_capex, kpi_param, f"{folder}mid_capex.pdf", legend=False, drop_boiler=True)
compare_gas_usage_bcm(n_scenarios_low_capex, kpi_param, f"{folder}low_capex.pdf", legend=False, drop_boiler=True)